### **task3_mobilenetv2_tinyimagenet**




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LOADING THE SHADOW.P DATASET

In [ ]:
#Loading the dataset shadow.p
import torch
import pickle

DATA_PATH = '/content/drive/MyDrive/AOML_PROJECT/mobilenetv2/shadow-002.p'
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    shadow_dataset = pickle.load(f)

dataloader = torch.utils.data.DataLoader(
    shadow_dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)

In [ ]:
print(type(shadow_dataset), len(shadow_dataset))

<class 'list'> 50000


TRAINING THE SHADOW MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


In [ ]:
#Custom Dataset for the shadow model
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label




In [ ]:
train_data, test_data = train_test_split(shadow_dataset, test_size=0.2, random_state=42)


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=70, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=70, shuffle=False, num_workers=2)
print("Length of train_data:", len(train_data))
print("Length of test_data:", len(test_data))

In [ ]:
# Defining the shadow model
shadow_model=models.mobilenet_v2(pretrained=True)
shadow_model.classifier[1] = nn.Linear(shadow_model.classifier[1].in_features, 200)  # Tiny ImageNet has 200 classes

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 35.6MB/s]


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(shadow_model.parameters(), lr=0.001)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
shadow_model.to(device)

In [ ]:
# Training the Shadow model
num_epochs = 15

for epoch in range(num_epochs):
    shadow_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = shadow_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

In [ ]:
# Evaluating the shadow model
shadow_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = shadow_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")



In [ ]:
# Save the model
torch.save(shadow_model, '/content/drive/MyDrive/AOML_PROJECT/shadow_model.pth')

# Download the model file
from google.colab import files
files.download('/content/drive/MyDrive/AOML_PROJECT/shadow_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shadow_model= torch.load("/content/drive/MyDrive/AOML_PROJECT/shadow_model.pth", map_location=torch.device('cpu'))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
shadow_model.to(device)

In [ ]:
shadow_model.eval()

GENERATING THE DATASET FOR THE ATTACK MODEL FROM THE SHADOW MODEL (MEMBER AND NON-MEMBER CONFIDENCE)


In [ ]:
import torch.nn.functional as F

attack_model_dataset = []

# Iterate over the test dataset (non-members)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = shadow_model(images)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
            attack_model_dataset.append(probabilities[i].tolist() + [0])  # 0 for non-member

In [ ]:
print(attack_model_dataset[0])
print(len(attack_model_dataset))

In [ ]:
# Iterate over the train dataset (members)
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = shadow_model(images)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
            attack_model_dataset.append(probabilities[i].tolist() + [1])  # 1 for member


In [ ]:
print(len(attack_model_dataset))

In [ ]:
import pickle
with open('/content/drive/MyDrive/AOML_PROJECT/attack_model_dataset.pkl','wb') as f:
  pickle.dump(attack_model_dataset, f)

In [ ]:
with open('/content/drive/MyDrive/AOML_PROJECT/attack_model_dataset.pkl', 'rb') as f:
  attack_model_dataset = pickle.load(f)

In [ ]:
print(attack_model_dataset[0])

ATTACK MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Convert the attack model dataset to tensors
attack_model_dataset = torch.tensor(attack_model_dataset, dtype=torch.float32)
X = attack_model_dataset[:, :-1]
y = attack_model_dataset[:, -1]

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


X_train_tensor = torch.tensor(X_train)
X_test_tensor = torch.tensor(X_test)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=80, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False)

In [ ]:
# Defining the attack model
class AttackModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


In [ ]:
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
attack_model = AttackModel(input_size, hidden_size, output_size)

criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.0001)


In [ ]:
# Training the attack model
num_epochs = 65
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attack_model.to(device)

for epoch in range(num_epochs):
    attack_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = attack_model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

In [ ]:
# Evaluating the attack model
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, labels in test_loader:
        tensors, labels = tensors.to(device), labels.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
       # print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 84.46%


In [ ]:
# Save the trained attack model
torch.save(attack_model.state_dict(), '/content/drive/MyDrive/AOML_PROJECT/attack_model.pth')


In [ ]:
attack_model= torch.load("/content/drive/MyDrive/AOML_PROJECT/mobilenetv2/attack_model_final.pth", map_location=torch.device('cpu'))

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/AOML_PROJECT/attack_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LOADING THE TARGET MODEL AND EVAL.P, GETTING THE CONFIDENCE_SCORE, USING THE CONFIDENCE SCORE TO EVALUVATE THE ATTACK MODEL

In [ ]:
import torch
import torchvision.models as models

MODEL_PATH = '/content/drive/MyDrive/AOML_PROJECT/mobilenetv2_tinyimagenet.pth'
# Change the MODEL_PATH to your local model path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = models.mobilenet_v2(num_classes=200).to(device)
# Change num_classes to 200 when you use the Tiny ImageNet dataset

state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])
# Test accuracy
acc = state_dict['acc']
# Training epoch (start from 0)
epoch = state_dict['epoch']

In [ ]:
import pickle
import torch

DATA_PATH = '/content/drive/MyDrive/AOML_PROJECT/mobilenetv2/eval.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    eval_dataset = pickle.load(f)
sample = next(iter(eval_dataset))
print(f'Sample structure: {sample}')
dataloader = torch.utils.data.DataLoader(
    eval_dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label, membership) in enumerate(dataloader):
    img = img.to(device)

In [ ]:
print(eval_dataset[:5])

In [ ]:
import pickle
import torch
import numpy as np
from PIL import Image

In [ ]:
confidence_scores = []
target_model.eval()
with torch.no_grad():
    for img, label, membership in dataloader:
        img = img.to(device)
        outputs = target_model(img)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        for i in range(len(img)):
          confidence_scores.append(probabilities[i].tolist()+[membership[i]])


In [ ]:
print(confidence_scores[0])

In [ ]:
class CustomDatasetConf(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx][:-1]
        label = self.data[idx][-1].item()
        return torch.tensor(tensor), label

In [ ]:
confidence_file_path = '/content/drive/MyDrive/AOML_PROJECT/confidence_score.pkl'
with open(confidence_file_path, 'wb') as f:
    pickle.dump(confidence_scores, f)

In [ ]:
with open('/content/drive/MyDrive/AOML_PROJECT/confidence_score.pkl', 'rb') as f:
  confidence_scores = pickle.load(f)

In [ ]:
conf_Score=CustomDatasetConf(confidence_scores)

In [ ]:
print(len(conf_Score))

In [ ]:
confidence_scores_loader = DataLoader(conf_Score, batch_size=64, shuffle=False)

In [ ]:
# Evaluating the attack model using thhe confidence score generated by the target model and eval.p
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, label in confidence_scores_loader:
        tensors, label = tensors.to(device), label.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += tensors.size(0)
        correct += (predicted==label).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 82.0%


GENERATING THE RESULT OF THE TEST.P

In [ ]:
import pickle
import torch

DATA_PATH = '/content/drive/MyDrive/AOML_PROJECT/mobilenetv2/test.p'
# Change the DATA_PATH to your local pickle file path

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    testp_dataset = pickle.load(f)

sample = next(iter(testp_dataset))
print(f'Sample structure: {sample}')
testp_dataloader = torch.utils.data.DataLoader(
    testp_dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label) in enumerate(testp_dataloader):
    img = img.to(device)

In [ ]:
print(testp_dataset[0][0].size())

In [ ]:
test_confidence_scores = []
target_model.eval()
with torch.no_grad():
    for img,label in testp_dataloader:
        img = img.to(device)
        outputs = target_model(img)
       # print(outputs.shape)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        for i in range(len(img)):
          test_confidence_scores.append(probabilities[i].tolist())
print(test_confidence_scores[0])

In [ ]:
print(len(test_confidence_scores[0]))

In [ ]:
class CustomDatasetConft(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx]

        return torch.tensor(tensor)

In [ ]:
test_conf_score=CustomDatasetConft(test_confidence_scores)

In [ ]:
print(test_conf_score[0])
print(len(test_conf_score[0]))

In [ ]:
# Evaluating the attack model using test.p
attack_model.eval()
Final_result=[]
with torch.no_grad():
    for tensors in test_conf_score:
        tensors = tensors.to(device)
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        Final_result.append(predicted.item())

RESULT

In [ ]:
predicted = np.array(Final_result)

In [ ]:
print(predicted)

[0. 0. 1. ... 1. 1. 0.]


In [ ]:
np.save('task3_mobilenetv2_tinyimagenet.npy', predicted)

In [ ]:
from google.colab import files
files.download('task3_mobilenetv2_tinyimagenet.npy')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>